In [2]:
# instruciton datasetの生成
import pandas as pd
import os
from datasets import load_dataset
import json
import random
from tqdm import tqdm

data_folder="data/0520orpo"

#dataフォルダ内をリセット
os.system(f"mkdir {data_folder}")
os.system(f"rm -rf {data_folder}/*")


ds_dict={}

def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"


records=[]

# # mixtralで自動生成したQ&A

score_threshold=4
ng_words=[
          #回答を避けるプロンプトの削除
          "申し訳","分からない","分かりません","すみません",
          #図表などへの言及
          "図","表",
          #日本関係の事項はハルシネーションが多いので消す
          "日本","京都","東京","寿司", 
          ]


mkdir: cannot create directory ‘data/0520orpo’: File exists


In [5]:
#マルチターン
ds=load_dataset("llm-jp/oasst1-21k-ja",)["train"]

records=[]
for record in ds:
    conversations=record["conversations"]
    if len(conversations)!=4:
        continue
    q=conversations[0]["value"]
    q+="\n### 応答:\n"+conversations[1]["value"]+"</s><s>\n### 指示:\n"+conversations[2]["value"]
    a=conversations[3]["value"]
    text=f"{question_template}{q}{answer_template}{a}"
    #records.append(text)
    d = {
        "question": q,
        "ref_answer": a,
        "dataset": "llm-jp/oasst1-21k-ja",
        "prompt": text,
    }
    records.append(d)
ds_dict["multi_turn_oasst"]=records
print(text),len(records)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
なぜ人工ニューロンはバイアス項を使うのか？重みを使うだけで十分ではないですか？
### 応答:
バイアスを導入することは、モデルに調整可能な閾値を持たせるために重要です。
例えばReLU活性化関数を持つニューロンを考えると、このニューロンが活性化関数の平坦領域で出力するかどうかは、その入力ベクトルのスケール不変条件のみに依存する可能性があります。
したがって、バイアスのかかったニューロンの非線型性は、バイアスのかかっていないニューロンよりもさらに広がる可能性があり、バイアスのかかったニューロンは「私の入力のこの組み合わせは3より大きい」というような客観的な基準を学習するが、バイアスのかかっていないニューロンは「私の入力のこの最初の組み合わせはこの2番目の組み合わせより大きい」というような相対的な基準しか学習しない可能性がある。</s><s>
### 指示:
平易な言葉で説明してもらえますか？人工ニューロンに詳しくない人のために？

### 応答:
ディープラーニングで使われているような人工ニューロンは、人間の脳の実際のニューロンの振る舞いをシミュレートするためのものです。人工ニューロンは入力を受け取り、処理し、出力を生成する。このシミュレーションでは、入力はニューロンへの接続の強さを表すように重み付けされ、バイアス項は、入力を受けなくても発火するニューロンの傾向を表す。

ニューロンを、入力を出力に変換する小さな機械と考えよう。重みを使って入力と出力を接続するだけなら、入力が十分に強ければ出力が得られるだけである。しかし現実の世界では、ニューロンは入力を受けなくても出力を出す傾向がある。そこでバイアスという用語が登場する。これは、入力が弱くても出力をオンにできる小さなスイッチのような働きをする。

言い換えれば、バイアス項は、出力が出る前に超えなければならない閾値のような働きをする。バイアス項を使うことで、ニューロンの振る舞いを微調整することができる。バイアス項がなければ、ニューロンは柔軟性を失い、強い入力を受 け取ったときだけ出力を出すように制限されてしまうだろう。


(None, 13100)

In [6]:

#マルチターン oasst 英語
ds=load_dataset("llm-jp/oasst2-33k-en",)["train"]

records=[]
for record in ds:
    conversations=record["conversations"]
    if len(conversations)!=4:
        continue
    q=conversations[0]["content"]
    q+="\n### 応答:\n"+conversations[1]["content"]+"</s><s>\n### 指示:\n"+conversations[2]["content"]
    a=conversations[3]["content"]
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
        "question": q,
        "ref_answer": a,
        "dataset": "llm-jp/oasst2-33k-en",
        "prompt": text,
    }
    records.append(d)
ds_dict["multi_turn_oasst_en"]=records
print(text),len(records)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
Four kids want to convince their parents to let them all play together outside but, since it's pretty late, their parents won't let them since they might wake up or disrupt their neighbors. The kids then decided to write a song, convincing their parents that they'll be quite outside. What song could've they come up with?
### 応答:
Verse 1:
We promise we'll be quiet,
We'll play outside without a riot.
We'll keep the volume low,
And not disturb the neighbors, you know.

Chorus:
Let us play, let us play,
We won't keep the neighbors up all day.
We'll have fun and make no noise,
We're good kids, we're not like those boys.

Verse 2:
We won't shout or scream,
We'll play nicely like a dream.
We'll stay close to the house,
And be quiet like a little mouse.

Chorus:
Let us play, let us play,
We won't keep the neighbors up all day.
We'll have fun and make no noise,
We're good kids, we're not like those boys.

Bridge:
We'll be outside for j

(None, 21560)

In [7]:
# マルチターン　自動生成

ds=load_dataset("kanhatakeyama/AutoMultiTurnByMixtral8x22b",)["train"]

records=[]
for record in ds:
    q=record["q1"]
    q+="\n### 応答:\n"+record["a1"]+"</s><s>\n### 指示:\n"+record["q2"]
    a=record["a2"]
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
        "question": q,
        "ref_answer": a,
        "dataset": "kanhatakeyama/AutoMultiTurnByMixtral8x22b",
        "prompt": text,
    }
    records.append(d)
ds_dict["multi_turn_mixtral"]=records
print(text),len(records)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
最近の忙しさの中で、自分が好きなフィットネスクラスに参加することが出来なくなってきました。仕事や家事、子育てのために毎日の時間はいっぱいで、スポーツを続けることが難しくなりました。しかし、健康は大切です。私たちはどのようにしてフィットネスクラスの習慣を取り組むべきでしょうか？
### 応答:
わかりました。時間が限られている状態でフィットネスクラスに参加するのは難しいですね。しかし、健康のために継続することは大切です。最初に考えられるのは、自分の時間表を調整してフィットネスクラスに参加する時間を確保することです。</s><s>
### 指示:
具体的にどのように時間表を調整すればいいのですか？

### 応答:
具体的な方法は、毎日のスケジュールを確認して、フィットネスクラスに参加する時間帯を決定することが重要です。例えば、仕事や家事、子育てのタイミングを調整して、毎週3回はフィットネスクラスに参加する時間帯を確保します。また、スケジュールが変更された場合は、予め代替日を決定しておくのも良い方法です。こうして時間表を調整することで、フィットネスクラスに参加する習慣を取り込むことが出来ます。


(None, 214139)

In [10]:

exclude_count=0

datasets=[
    #load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train"),

    load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train"),
    load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA-CC",split="train"),
    load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA-other",split="train"),
    load_dataset("kanhatakeyama/OrcaJaMixtral8x22b",split="train"),
    load_dataset("kanhatakeyama/ChatbotArenaJaMixtral8x22b",split="train"),

]
for dataset in datasets:
    for original_record in tqdm(iter(dataset)):
        q=clean_autogen(original_record["question"])
        a=clean_autogen(original_record["answer"])
        if q=="" or a=="":
            continue

        if "score" in original_record:
            if original_record["score"] is None:
                continue
            if int(original_record["score"])<score_threshold:
                continue


        exclude_flag=False

        #回答しないパターンのrecordを除外
        for ng_word in ng_words:
            if a.find(ng_word)>=0 or q.find(ng_word)>=0:
                #print("excluded:",a)
                exclude_flag=True
                exclude_count+=1
                continue

        if exclude_flag:
            continue
        #if len(a)<10:
        #    print("too short answer",a)
        #    continue

        text=f"{question_template}{q}{answer_template}{a}"
        if a!="":
            d = {
                "question": q,
                "ref_answer": a,
                "dataset": original_record["database"],
                "prompt": text,
            }
            records.append(d)

ds_dict["auto_gen_mixtral"]=records


212355it [00:08, 26468.08it/s]
53950it [00:01, 27828.80it/s]
21899it [00:00, 28039.84it/s]
734876it [00:36, 20088.37it/s]
17354it [00:00, 20917.93it/s]


In [11]:

# %% [markdown]
# # hachiさんのalpaca + mixtral dataset

# %%

hachi_datasets={
   "HachiML/Hachi-Alpaca": load_dataset("HachiML/Hachi-Alpaca",split='v1.0_cleaned'),
    "HachiML/Evol-Alpaca-gen3-500":load_dataset("HachiML/Evol-Alpaca-gen3-500",split='train'),
}

# %%
records=[]
for key,hachi_ds in hachi_datasets.items():
    for record in tqdm(hachi_ds):
        q=record["instruction"]
        if "input" in record:
            inp=record["input"]
        else:
            inp=""
        if inp!="":
            q+="\n"+inp
        a=record["output"]
        if q=="" or a=="":
            continue
        text=f"{question_template}{q}{answer_template}{a}"
        d = {
                "question": q,
                "ref_answer": a,
                "dataset": key,
                "prompt": text,
            }
        records.append(d)

    ds_dict[f"hachi_{key}"]=records


100%|██████████| 507/507 [00:00<00:00, 18038.26it/s]


In [12]:

# %% [markdown]
# # Bumpo dataset

#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
# %%
records=[]
for original_record in iter(ds2):
    q=(original_record["question"])
    a=(original_record["answer"])
    inst=(original_record["instruction"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
            "question": q,
            "ref_answer": a,
            "dataset": "hatakeyama-llm-team/BumpoRikai",
            "prompt": text,
        }
    records.append(d)
ds_dict["bumpo_rikai"]=records
records[1]



{'question': 'faカップ準決勝に延長戦はありますか',
 'ref_answer': 'YES',
 'dataset': 'hatakeyama-llm-team/BumpoRikai',
 'prompt': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nfaカップ準決勝に延長戦はありますか\n\n### 応答:\nYES'}

In [22]:

#minnade
m_ds=load_dataset("minnade/chat-daily",split="train")

id_to_content={}
for record in m_ds:
    id_to_content[record["id"]]=record["body"]

questions=[]
for record in m_ds:
    if record["role"]=="assistant":
        q=id_to_content[record["parent_id"]]
        a=record["body"]
        if a is None:
            continue
        if len(a)<4:
            continue
        #questions.append((q,a))
        text=f"{question_template}{q}{answer_template}{a}"
        d = {
                "question": q,
                "ref_answer": a,
                "dataset": "hatakeyama-llm-team/BumpoRikai",
                "prompt": text,
            }
        records.append(d)
ds_dict["minnade"]=questions

In [23]:


# %%
all_recrds=[]
for k,v in ds_dict.items():
    all_recrds+=v

# %%

def write_jsonl(records,
    output_path="data/all.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    random.shuffle(records)
    df=pd.DataFrame.from_dict(records[:-n_eval][:n_train])
    df["text"]=df["prompt"].astype(str)
    df=df.drop("prompt",axis=1)
    df=df.reset_index()
    df.to_parquet(output_path)
    
    #eval
    df=pd.DataFrame.from_dict(records[-n_eval:])
    df["text"]=df["prompt"].astype(str)
    df=df.drop("prompt",axis=1)
    df=df.reset_index()
    df.to_parquet(output_path.replace(".parquet","_eval.parquet"))
    return df

In [24]:
n_train=10**10
#df=write_jsonl(all_recrds,f"{data_folder}/all_{n_train}.parquet",n_train=n_train)

# code dataset

In [25]:
code_ds_dict={}

In [26]:
#openmathinst 数GB
# 思考が硬直化してそうなので、やめる
"""
openmath_ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train")

records=[]
for original_record in iter(openmath_ds):
    q=(original_record["question_ja"])
    a=(original_record["generated_solution_ja"])
    #inst=(original_record["instruction"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["openmathja"]=records
records[1]
"""

'\nopenmath_ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train")\n\nrecords=[]\nfor original_record in iter(openmath_ds):\n    q=(original_record["question_ja"])\n    a=(original_record["generated_solution_ja"])\n    #inst=(original_record["instruction"])\n    if q=="" or a=="":\n        continue\n    text=f"{question_template}{q}{answer_template}{a}"\n    records.append(text)\ncode_ds_dict["openmathja"]=records\nrecords[1]\n'

In [27]:
#code 50k
ds=load_dataset("HachiML/alpaca_jp_python",split="v1.0_cleaned")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
            "question": q,
            "ref_answer": a,
            "dataset": "HachiML/alpaca_jp_python",
            "prompt": text,
        }
    records.append(d)
code_ds_dict["code_hachi"]=records
records[1]



Generating _archive_v0.1 split: 100%|██████████| 8629/8629 [00:00<00:00, 127036.33 examples/s]


{'question': '与えられたリスト内の重複する値を除去するPythonスクリプトを書きなさい。\n```python\nnumbers = [1, 2, 2, 3, 4, 4, 5]\n```',
 'ref_answer': '```python\nunique_numbers = list(set(numbers))\nprint(unique_numbers)\n```',
 'dataset': 'HachiML/alpaca_jp_python',
 'prompt': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n与えられたリスト内の重複する値を除去するPythonスクリプトを書きなさい。\n```python\nnumbers = [1, 2, 2, 3, 4, 4, 5]\n```\n\n### 応答:\n```python\nunique_numbers = list(set(numbers))\nprint(unique_numbers)\n```'}

In [28]:
#code 5k
ds=load_dataset("kunishou/amenokaku-code-instruct",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
            "question": q,
            "ref_answer": a,
            "dataset": "kunishou/amenokaku-code-instruct",
            "prompt": text,
        }
    records.append(d)
code_ds_dict["amenokaku"]=records
records[1]



{'question': 'pandasでレシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。',
 'ref_answer': "import pandas as pd\n\ndf_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)",
 'dataset': 'kunishou/amenokaku-code-instruct',
 'prompt': "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\npandasでレシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。\n\n### 応答:\nimport pandas as pd\n\ndf_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)"}

In [29]:
#算数 0.5k
ds=load_dataset("saldra/sakura_japanese_dataset",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
            "question": q,
            "ref_answer": a,
            "dataset": "saldra/sakura_japanese_dataset",
            "prompt": text,
        }
    records.append(d)
code_ds_dict["sakura"]=records
records[1]



{'question': '小王は、20％の薬を含む農薬150kgを、5％の薬を含む溶液に希釈したいと考えています。 何kgの水を加えればよいでしょうか。',
 'ref_answer': 'x=150*20%/5%-150 よって答えは 450キログラム',
 'dataset': 'saldra/sakura_japanese_dataset',
 'prompt': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n小王は、20％の薬を含む農薬150kgを、5％の薬を含む溶液に希釈したいと考えています。 何kgの水を加えればよいでしょうか。\n\n### 応答:\nx=150*20%/5%-150 よって答えは 450キログラム'}

In [17]:
# meta math
#数学的な基礎力が低く、勉強してもあまり意味がなさそう
"""
ds=load_dataset("meta-math/MetaMathQA",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["query"])
    a=(original_record["response"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["meta_math"]=records
records[1]

"""

'\nds=load_dataset("meta-math/MetaMathQA",split="train")\n\nrecords=[]\nfor original_record in iter(ds):\n    q=(original_record["query"])\n    a=(original_record["response"])\n    if q=="" or a=="":\n        continue\n    text=f"{question_template}{q}{answer_template}{a}"\n    records.append(text)\ncode_ds_dict["meta_math"]=records\nrecords[1]\n\n'

In [18]:
"""
ds=load_dataset("microsoft/orca-math-word-problems-200k",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["question"])
    a=(original_record["answer"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["orca_math"]=records
records[1]
"""


'\nds=load_dataset("microsoft/orca-math-word-problems-200k",split="train")\n\nrecords=[]\nfor original_record in iter(ds):\n    q=(original_record["question"])\n    a=(original_record["answer"])\n    if q=="" or a=="":\n        continue\n    text=f"{question_template}{q}{answer_template}{a}"\n    records.append(text)\ncode_ds_dict["orca_math"]=records\nrecords[1]\n'

In [30]:

#python codes 150k
ds=load_dataset("m-a-p/CodeFeedback-Filtered-Instruction",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["query"])
    a=(original_record["answer"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    d = {
            "question": q,
            "ref_answer": a,
            "dataset": "m-a-p/CodeFeedback-Filtered-Instruction",
            "prompt": text,
        }
    records.append(d)
code_ds_dict["codefeedback"]=records
records[1]



{'question': "Write a function to find the number of distinct states in a given matrix. Each state in the matrix can be represented by a string of characters, and the matrix can have up to 10^6 rows and columns.\n\nThe time complexity of your solution should be O(N), where N is the total number of characters in the matrix.\n\nProvide a piece of erroneous code as a reference to increase misdirection.\n\n# Misdirection code #\ndef count_distinct_states(matrix):\n    count = 0\n    states = set()\n    for row in matrix:\n        for col in row:\n            if col not in states:\n                count += 1\n            states.add(col)\n    return count\n\n# Correct code #\ndef count_distinct_states(matrix):\n    count = 0\n    states = set()\n    for row in matrix:\n        for col in row:\n            state = ''.join(col)\n            if state not in states:\n                count += 1\n            states.add(state)\n    return count\n\nmatrix = [['A', 'B', 'C'],\n          ['A', 'B', 'D

In [31]:
# %%
all_code_records=[]
for k,v in code_ds_dict.items():
    all_code_records+=v
len(all_code_records)

173360

In [32]:

n_train=10**10
_=write_jsonl(all_recrds+all_code_records,f"{data_folder}/code_all_{n_train}.parquet",n_train=n_train)

In [33]:
len(all_recrds)

1898089